# Descripción del proyecto

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

# Instrucciones del proyecto
1. Descarga y examina los datos.
2. Entrena diferentes modelos con varios hiperparámetros (debes hacer al menos dos modelos diferentes, pero más es mejor. Recuerda, varias implementaciones de potenciación del gradiente no cuentan como modelos diferentes). El punto principal de este paso es comparar métodos de potenciación del gradiente con bosque aleatorio, árbol de decisión y regresión lineal.
3. Analiza la velocidad y la calidad de los modelos.

**Observaciones:**

* Utiliza la métrica RECM para evaluar los modelos.
* La regresión lineal no es muy buena para el ajuste de hiperparámetros, pero es perfecta para hacer una prueba de cordura de otros métodos. Si la potenciación del gradiente funciona peor que la regresión lineal, definitivamente algo salió mal.
* Aprende por tu propia cuenta sobre la librería LightGBM y sus herramientas para crear modelos de potenciación del gradiente (gradient boosting).
* Idealmente, tu proyecto debe tener regresión lineal para una prueba de cordura, un algoritmo basado en árbol con ajuste de hiperparámetros (preferiblemente, bosque aleatorio), LightGBM con ajuste de hiperparámetros (prueba un par de conjuntos), y CatBoost y XGBoost con ajuste de hiperparámetros (opcional).
* Toma nota de la codificación de características categóricas para algoritmos simples. LightGBM y CatBoost tienen su implementación, pero XGBoost requiere OHE.
* Puedes usar un comando especial para encontrar el tiempo de ejecución del código de celda en Jupyter Notebook. Encuentra ese comando.
* Dado que el entrenamiento de un modelo de potenciación del gradiente puede llevar mucho tiempo, cambia solo algunos parámetros del modelo.
* Si Jupyter Notebook deja de funcionar, elimina las variables excesivas por medio del operador del:
<div class="alert alert-block alert-info">
    <b>del features_train</b>
</div>  

# Descripción de los datos
El dataset está almacenado en el archivo /datasets/car_data.csv. descargar dataset.

**Características**

* **DateCrawled** — fecha en la que se descargó el perfil de la base de datos
* **VehicleType** — tipo de carrocería del vehículo
* **RegistrationYear** — año de matriculación del vehículo
* **Gearbox** — tipo de caja de cambios
* **Power** — potencia (CV)
* **Model** — modelo del vehículo
* **Mileage** — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)
* **RegistrationMonth** — mes de matriculación del vehículo
* **FuelType** — tipo de combustible
* **Brand** — marca del vehículo
* **NotRepaired** — vehículo con o sin reparación
* **DateCreated** — fecha de creación del perfil
* **NumberOfPictures** — número de fotos del vehículo
* **PostalCode** — código postal del propietario del perfil (usuario)
* **LastSeen** — fecha de la última vez que el usuario estuvo activo
* **Objetivo**
  * Price — precio (en euros)

## Preparación de datos

In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors


from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import lightgbm as lgb
from catboost import CatBoostClassifier
from xgboost import XGBClassifier



In [2]:
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
# Revisando los datos 
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
#Aplicado snake_case a las columnas  
df.columns = (df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())
df.head(3)

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47


In [5]:
#  eliminacios de columnas irrelevantes para las predicciones 
df = df.drop(['date_crawled', 'registration_year','registration_month','date_created','number_of_pictures','postal_code','last_seen'], axis=1)

In [6]:
# revision de valores duplicados 
print('Se encontraron un total de valores duplicados:',df.duplicated().sum())


Se encontraron un total de valores duplicados: 73748


In [7]:
#Eliminacion de valores duplicados 
df = df.drop_duplicates().reset_index(drop=True)
print('Se encontraron un total de valores duplicados:',df.duplicated().sum())

Se encontraron un total de valores duplicados: 0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280621 entries, 0 to 280620
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   price         280621 non-null  int64 
 1   vehicle_type  247978 non-null  object
 2   gearbox       264547 non-null  object
 3   power         280621 non-null  int64 
 4   model         263002 non-null  object
 5   mileage       280621 non-null  int64 
 6   fuel_type     251307 non-null  object
 7   brand         280621 non-null  object
 8   not_repaired  220575 non-null  object
dtypes: int64(3), object(6)
memory usage: 19.3+ MB


In [9]:
# revisando los valores ausentes

print('se muestran los valores ausentes en df')
100*df.isna().sum()/df.shape[0]

se muestran los valores ausentes en df


price            0.000000
vehicle_type    11.632415
gearbox          5.728010
power            0.000000
model            6.278575
mileage          0.000000
fuel_type       10.446118
brand            0.000000
not_repaired    21.397543
dtype: float64

In [10]:
#df['not_repaired'].fillna('unknown',inplace=True)
df['not_repaired'].fillna('unknown',inplace=True)

In [11]:
# eliminar los valores ausentes 
df = df.dropna().reset_index(drop=True)
df.isna().sum()

price           0
vehicle_type    0
gearbox         0
power           0
model           0
mileage         0
fuel_type       0
brand           0
not_repaired    0
dtype: int64

In [12]:
#Revision de tipos de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218162 entries, 0 to 218161
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   price         218162 non-null  int64 
 1   vehicle_type  218162 non-null  object
 2   gearbox       218162 non-null  object
 3   power         218162 non-null  int64 
 4   model         218162 non-null  object
 5   mileage       218162 non-null  int64 
 6   fuel_type     218162 non-null  object
 7   brand         218162 non-null  object
 8   not_repaired  218162 non-null  object
dtypes: int64(3), object(6)
memory usage: 15.0+ MB


In [13]:
df.head(5)

,price,vehicle_type,gearbox,power,model,mileage,fuel_type,brand,not_repaired
0,9800,suv,auto,163,grand,125000,gasoline,jeep,unknown
1,1500,small,manual,75,golf,150000,petrol,volkswagen,no
2,3600,small,manual,69,fabia,90000,gasoline,skoda,no
3,650,sedan,manual,102,3er,150000,petrol,bmw,yes
4,2200,convertible,manual,109,2_reihe,150000,petrol,peugeot,no


In [14]:
#Cambiando el tipo de dato para mejorar la manipulacion de los datos 
type = ['vehicle_type','gearbox','model','fuel_type','brand','not_repaired']

for x in type:
    df[x] = df[x].astype('category')


In [15]:
#Revisando los cambios efectuados 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218162 entries, 0 to 218161
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   price         218162 non-null  int64   
 1   vehicle_type  218162 non-null  category
 2   gearbox       218162 non-null  category
 3   power         218162 non-null  int64   
 4   model         218162 non-null  category
 5   mileage       218162 non-null  int64   
 6   fuel_type     218162 non-null  category
 7   brand         218162 non-null  category
 8   not_repaired  218162 non-null  category
dtypes: category(6), int64(3)
memory usage: 6.5 MB


In [16]:
df.describe()

,price,power,mileage
count,218162.000000,218162.000000,218162.000000
mean,5195.876940,121.085890,125008.365343
std,4780.188855,189.319423,38629.723463
min,0.000000,0.000000,5000.000000
25%,1499.000000,75.000000,100000.000000
50%,3500.000000,113.000000,150000.000000
75%,7600.000000,150.000000,150000.000000
max,20000.000000,20000.000000,150000.000000


In [17]:
df.price.median()

3500.0

In [18]:
df.loc[df['price'] > 3500, 'select_price'] = 1
df.loc[df['price'] <= 13500, 'select_price'] = 0

In [20]:
df

,price,vehicle_type,gearbox,power,model,mileage,fuel_type,brand,not_repaired,select_price
0,9800,suv,auto,163,grand,125000,gasoline,jeep,unknown,0.0
1,1500,small,manual,75,golf,150000,petrol,volkswagen,no,0.0
2,3600,small,manual,69,fabia,90000,gasoline,skoda,no,0.0
3,650,sedan,manual,102,3er,150000,petrol,bmw,yes,0.0
4,2200,convertible,manual,109,2_reihe,150000,petrol,peugeot,no,0.0
...,...,...,...,...,...,...,...,...,...,...
218157,3999,wagon,manual,3,3er,150000,gasoline,bmw,no,0.0
218158,3200,sedan,manual,225,leon,150000,petrol,seat,yes,0.0
218159,1199,convertible,auto,101,fortwo,125000,petrol,smart,no,0.0
218160,9200,bus,manual,102,transporter,150000,gasoline,volkswagen,no,0.0


## Entrenamiento del modelo 

In [22]:
# transformando 
feature_names = ['power','mileage']

transformer_mas = MaxAbsScaler()

df_scaled = df.copy()
df_scaled[feature_names] = transformer_mas.fit_transform(df[feature_names].to_numpy())
print(df_scaled)

        price vehicle_type gearbox    power        model   mileage fuel_type  \
0        9800          suv    auto  0.00815        grand  0.833333  gasoline   
1        1500        small  manual  0.00375         golf  1.000000    petrol   
2        3600        small  manual  0.00345        fabia  0.600000  gasoline   
3         650        sedan  manual  0.00510          3er  1.000000    petrol   
4        2200  convertible  manual  0.00545      2_reihe  1.000000    petrol   
...       ...          ...     ...      ...          ...       ...       ...   
218157   3999        wagon  manual  0.00015          3er  1.000000  gasoline   
218158   3200        sedan  manual  0.01125         leon  1.000000    petrol   
218159   1199  convertible    auto  0.00505       fortwo  0.833333    petrol   
218160   9200          bus  manual  0.00510  transporter  1.000000  gasoline   
218161   3400        wagon  manual  0.00500         golf  1.000000  gasoline   

             brand not_repaired  select

#divicion de los datos 
features = df_scaled.drop(['price'])

In [23]:
df_ohe = pd.get_dummies(df_scaled, drop_first=True)

In [25]:
df_ohe.shape

(218162, 307)

In [26]:
features = df_ohe.drop(['price','select_price'], axis=1)
target = df_ohe['select_price']

In [27]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size= .25, random_state=12345 )
print(features_train.shape)
print(features_valid.shape)
print(target_train.shape)
print(target_valid.shape)




(163621, 305)
(54541, 305)
(163621,)
(54541,)


### Modelos sin  potenciación del gradiente

#### regresion lineal 

In [30]:
# inicio de tiempo de entrenamiento
start_time_train = time.time()

model = LinearRegression()
model.fit(features_train, target_train)

#tiempo de entrenamiento
time_train = time.time() -start_time_train

# inicio de tiempo de prediccion 
start_time_predic= time.time()

predicted_valid = model.predict(features_valid)

#tiempo de prediccion
time_predic = time.time() -start_time_predic

# resultados obtenidos 
recm = mean_squared_error(target_valid, predicted_valid)**0.5

print('Linear Regression')
print(f'La calidad de la prediccion es RECM es de {recm:.4f}')

# Calcular el tiempo de ejecución

print(f'Tiempo de ejecución del entrenamiento : {time_train:.4f} segundos')
print(f'Tiempo de ejecución de la prediccion: {time_predic:.4f} segundos')


Linear Regression
La calidad de la prediccion es RECM es de 0.2417
Tiempo de ejecución del entrenamiento : 5.3412 segundos
Tiempo de ejecución de la prediccion: 0.1015 segundos


#### bosque aleatorio, 

In [26]:

best_result = 10000
best_est = 0
# selecciona el rango del hiperparámetro
for est in range(1, 20):
    
    # inicio de tiempo de entrenamiento
    start_time_train = time.time()
    model = RandomForestClassifier(min_samples_split=5, min_samples_leaf =1,class_weight='balanced',
                                   random_state=12345, n_estimators=est,max_depth=4, criterion='gini')  
    model.fit(features_train, target_train ) 
    # #tiempo de entrenamiento
    time_train = time.time() -start_time_train


    # inicio de tiempo de prediccion 
    start_time_predic = time.time()
    
    
    predicted_valid = model.predict(features_valid)
    # #tiempo de prediccion
    # time_predic = time.time() -start_time_predic
    
    
    result = mean_squared_error(target_valid, predicted_valid)**0.5# calcula la RECM en el conjunto de validación
    if result < best_result:
        best_result = result
        best_est = est


# resultados obtenidos 

print('Modelo RandomForestClassifier')
print(f'La calidad de la prediccion de  RECM: (n_estimators = {best_est}): {best_result:.4f}')

# Calcular el tiempo de ejecución

print(f'Tiempo de ejecución del entrenamiento : {time_train:.4f} segundos')
print(f'Tiempo de ejecución de la prediccion: {time_predic:.4f} segundos')

Modelo RandomForestClassifier
La calidad de la prediccion de  RECM: (n_estimators = 17): 0.4661
Tiempo de ejecución del entrenamiento : 0.8174 segundos
Tiempo de ejecución de la prediccion: 0.0990 segundos


#### árbol de decisión

In [27]:
best_result = 10000
best_depth = 0
# selecciona el rango del hiperparámetro
for depth in range(5,25):
    # inicio de tiempo de entrenamiento
    start_time_train = time.time()

    model = DecisionTreeRegressor(random_state=12345,max_depth=depth)
    model.fit(features_train, target_train) 
    #tiempo de entrenamiento
    time_train = time.time() - start_time_train

     # inicio de tiempo de prediccion 
    start_time_predic = time.time()

    predictions_valid = model.predict(features_valid) 
    
    #tiempo de prediccion
    time_predic = time.time() -start_time_predic

    
    result = mean_squared_error(target_valid, predictions_valid)**0.5
    if result < best_result:
        best_result = result
        best_depth = depth
        
# resultados obtenidos 
 
print('Modelo Arbol de  decisiones')
print(f'La calidad de la prediccion es RECM es de (max_depth = {best_depth}): {best_result:.4f}')

#  Calcular el tiempo de ejecución

print(f'Tiempo de ejecución del entrenamiento : {time_train:.4f} segundos')
print(f'Tiempo de ejecución de la prediccion: {time_predic:.4f} segundos')

Modelo Arbol de  decisiones
La calidad de la prediccion es RECM es de (max_depth = 15): 0.2100
Tiempo de ejecución del entrenamiento : 3.2559 segundos
Tiempo de ejecución de la prediccion: 0.0426 segundos


### Modelos de  potenciación del gradiente

####  XGBoost

In [32]:
# inicio de tiempo de entrenamiento
start_time_train = time.time()
 
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss',n_estimators=10, seed=12345 ,use_label_encoder=False)
model.fit(features_train, target_train)

#tiempo de entrenamiento
time_train = time.time() -start_time_train


# inicio de tiempo de prediccion 
start_time_predic= time.time()

predicted_valid= model.predict(features_valid)

#tiempo de prediccion
time_predic = time.time() -start_time_predic

# Muestra de resultados obtenidos 
recm = mean_squared_error(target_valid, predicted_valid)**0.5
print('Modelo XGBoost')
print(f"La calidad de la prediccion es RECM es de {recm:.4f}")

# Calcular el tiempo de ejecución

print(f"Tiempo de ejecución del entrenamiento : {time_train:.4f} segundos")
print(f"Tiempo de ejecución de la prediccion: {time_predic:.4f} segundos")

Modelo XGBoost
La calidad de la prediccion es RECM es de 0.2478
Tiempo de ejecución del entrenamiento : 21.6321 segundos
Tiempo de ejecución de la prediccion: 0.3135 segundos


#### LightGBM

In [29]:

# inicio de tiempo de entrenamiento
start_time_train = time.time()


# Crear el modelo LightGBM
model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', n_estimators=100, seed=12345)
model.fit(features_train, target_train)
#tiempo de entrenamiento
time_train = time.time() -start_time_train


 # inicio de tiempo de prediccion 
start_time_predic = time.time()

predicted_valid = model.predict(features_valid)

#tiempo de prediccion
time_predic = time.time() -start_time_predic

recm = mean_squared_error(target_valid, predicted_valid) ** 0.5

print('Modelo LightGBM')
print(f'La calidad de la prediccion es RECM es de {recm:.4f}')

#  Calcular el tiempo de ejecución

print(f'Tiempo de ejecución del entrenamiento : {time_train:.4f} segundos')
print(f'Tiempo de ejecución de la prediccion: {time_predic:.4f} segundos')


Modelo LightGBM
La calidad de la prediccion es RECM es de 0.2298
Tiempo de ejecución del entrenamiento : 3.0813 segundos
Tiempo de ejecución de la prediccion: 0.4099 segundos


#### CatBoost

In [30]:

# inicio de tiempo de entrenamiento
start_time_train = time.time()


model = CatBoostClassifier(loss_function="Logloss", iterations=300,random_seed=12345)
model.fit(features_train, target_train, verbose=30)

#tiempo de entrenamiento
time_train = time.time() -start_time_train


 # inicio de tiempo de prediccion 
start_time_predic = time.time()

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

#tiempo de prediccion
time_predic = time.time() -start_time_predic


# resultados obtenidos 
recm = mean_squared_error(target_valid, probabilities_one_valid)**0.5
print('Modelo CatBoost')
print(f'La calidad de la prediccion es RECM es de {recm:.4f}')

#  Calcular el tiempo de ejecución

print(f'Tiempo de ejecución del entrenamiento : {time_train:.4f} segundos')
print(f'Tiempo de ejecución de la prediccion: {time_predic:.4f} segundos')



Learning rate set to 0.273983
0:	learn: 0.3455138	total: 73.4ms	remaining: 21.9s
30:	learn: 0.1458106	total: 782ms	remaining: 6.79s
60:	learn: 0.1353142	total: 1.49s	remaining: 5.85s
90:	learn: 0.1295587	total: 2.2s	remaining: 5.05s
120:	learn: 0.1251322	total: 2.91s	remaining: 4.3s
150:	learn: 0.1219474	total: 3.62s	remaining: 3.57s
180:	learn: 0.1192052	total: 4.33s	remaining: 2.85s
210:	learn: 0.1170036	total: 5.04s	remaining: 2.13s
240:	learn: 0.1152637	total: 5.77s	remaining: 1.41s
270:	learn: 0.1134899	total: 6.48s	remaining: 693ms
299:	learn: 0.1119467	total: 7.16s	remaining: 0us
Modelo CatBoost
La calidad de la prediccion es RECM es de 0.1916
Tiempo de ejecución del entrenamiento : 7.9074 segundos
Tiempo de ejecución de la prediccion: 0.0368 segundos


## Análisis del modelo

In [31]:


# validacion de datos
data_valid = pd.DataFrame() 
 
data_valid['modelo'] = ['linear_regression','Random_forestClassifier', 'Decision_tree_classifier','XGBoost','LightGBM','CatBoost']
data_valid['recm'] = [0.2417,0.4660,0.2100,0.0614,0.2298,0.1916]
data_valid['tiempo_entrenamiento'] = [6.1554,0.8246,3.2771,21.8428 ,3.0501 ,7.8994]
data_valid['tiempo_prediccion'] = [0.1024 ,0.1011,0.0419,0.3486,0.4150,0.0369]


data_valid

,modelo,recm,tiempo_entrenamiento,tiempo_prediccion
0,linear_regression,0.2417,6.1554,0.1024
1,Random_forestClassifier,0.4660,0.8246,0.1011
2,Decision_tree_classifier,0.2100,3.2771,0.0419
3,XGBoost,0.0614,21.8428,0.3486
4,LightGBM,0.2298,3.0501,0.4150
5,CatBoost,0.1916,7.8994,0.0369


In [32]:
# Revisando que valor de redm tiene mejor calidad 
data_valid[data_valid['recm'] == (data_valid['recm'].min())]

,modelo,recm,tiempo_entrenamiento,tiempo_prediccion
3,XGBoost,0.0614,21.8428,0.3486


In [33]:
# Revisando mejor tiempo entrenamiento
data_valid[data_valid['tiempo_entrenamiento'] == (data_valid['tiempo_entrenamiento'].min())]

,modelo,recm,tiempo_entrenamiento,tiempo_prediccion
1,Random_forestClassifier,0.466,0.8246,0.1011


In [34]:
# Revisando mejor tiempo prediccion
data_valid[data_valid['tiempo_prediccion'] == (data_valid['tiempo_prediccion'].min())]

,modelo,recm,tiempo_entrenamiento,tiempo_prediccion
5,CatBoost,0.1916,7.8994,0.0369


### Observaciones

Se observo que el mejor modelo para la predicciones de los datos es XGBoost aunque en tiempo de entrenamiento y prediccion no es  el mas efiente sus resultado es el mejor en comparacion con los otro modelos, en cuanto mejor tiempo de entrenamiento es el modelo de  Random forestClassifier y mejor tiempo de prediccion es el modelo de  CatBoost, con lo que concluimos que el utilizar los modelos de potenciación del gradiente se obtienen mejor resultados 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores
- [x]  Las celdas con el código han sido colocadas en orden de ejecución
- [x]  Los datos han sido descargados y preparados
- [x]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos